In [1]:
from zplot import zplot,zplot_ud
from zdata import Stock_indicators_x
import pandas as pd
import numpy as np
import time
import plotly.plotly as py
# py.sign_in('pjl88209', 'a0hVGm4ROxyhXEXlVY5m')
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool

import plotly.tools as tls
import cufflinks as cf


from sklearn.ensemble import RandomForestClassifier

from sklearn.cross_validation import cross_val_score
from sklearn.datasets import make_blobs

# meta-estimator
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier 

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import TimeSeriesSplit 




# print (cf.__version__)

/srv/env/lib64/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



# 0. data preparation Done

In [2]:
st=index_components('399004.XSHE')
len(index_components('399004.XSHE'))

100

In [3]:
st=st[68:]

In [4]:
df_out=pd.DataFrame(columns=['stock_name','clf_name','predict_up_truely_up/predict_up','predict_up','predict_up_truely_up','actually_uprise'])

df_out



,stock_name,clf_name,predict_up_truely_up/predict_up,predict_up,predict_up_truely_up,actually_uprise


In [5]:
for one_stock_name in st:

    random_stock=st[np.random.randint(1,len(st)+1)]

    s=Stock_indicators_x(one_stock_name,start='2015-6-01')

    s.indicators.shape

    s.indicators.columns

    si=s.indicators.copy()

    si=si.iloc[:,:14]
    si.shape

    ## 0.1 shift tomorrow price back - time travel

    # key point!!!
    train_test_database=si.join(si[['close','high']].shift(-1),rsuffix='_tomorrow')

    train_test_database=train_test_database.dropna()

    threshold=0.001

    t1=(train_test_database.close_tomorrow-train_test_database.close)/train_test_database.close
    t2=(train_test_database.high_tomorrow-train_test_database.high)/train_test_database.high

    t1.name='close_threshold'
    t2.name='high_threshold'

    train_test_database =train_test_database.join(t1).join(t2)

    train_test_database.columns

    t_f=train_test_database[['close_threshold','high_threshold']].apply(lambda x :  x>threshold )


    train_test_database=train_test_database.join(t_f,rsuffix='_TF')




    final_t_f=train_test_database.close_threshold_TF & train_test_database.high_threshold_TF
    final_t_f.name='final_t_f'

    train_test_database=train_test_database.join(final_t_f,rsuffix='_final_t_f')





    data=si.join(final_t_f)



    def replace(x): # replace  true or false with 1 ,0
        if x  :
            return 1
        else:
            return -1

    data['label']=data.final_t_f.apply(lambda x: replace(x))

    positive_percentage=data['label'] [data['label']==1].count()/data['label'].count()
    positive_percentage=round(positive_percentage,5)



    data.SMA10=data.SMA10>data.close

    data.WMA=data.WMA>data.close

    def mom_j(x):
        if x>0:
            return True
        else:
            return False
    data.MOM=data.MOM.apply(lambda x: mom_j(x))

    data.Sto_k=data.Sto_k.diff()[1:].apply(lambda x: x>0)

    data.Sto_d=data.Sto_d.diff()[1:].apply(lambda x: x>0)

    x=data.RSI<=30
    y1=((30<data.RSI  ) & ( data.RSI <70 ))
    y2=data.RSI.diff().rename('RSI_diff')>0
    data.RSI=x | (y1 &y2 )

    data.MACD=data.MACD.diff()[1:]>0

    data.Willi_R=data.Willi_R.diff()[1:]>0

    data.AD=data.AD.diff()[1:]>0

    x=data.CCI>=200
    y1=((150<data.CCI  ) & ( data.CCI <200 ))
    y2=data.CCI.diff().rename('RSI_diff')>0
    data.CCI=x | (y1 &y2 )

    data=data.dropna()



    def t_f2val(x):  #translate true or false to values of 0,1
        if x:
            return 1
        else :
            return 0
    data_X=data.iloc[:,:10].applymap(lambda x :  t_f2val(x))
    data_y=data.iloc[:,-1]

    data_X_y=data_X.join(data_y)

    data_X_y.shape













    len(data_X_y)

    moving_test_times_of_future=33

    in_sample=data_X_y[:-moving_test_times_of_future]

    out_of_sample=data_X_y[-moving_test_times_of_future:]

    insample_data_label_positive=in_sample[in_sample['label']==1]



    xy1=insample_data_label_positive

    len(xy1)

    insample_data_label_nagative=in_sample[in_sample['label']==-1]



    xy2=insample_data_label_nagative[-int(len(xy1)*1.5):]
    len(xy2)

    new_data=pd.concat([xy1,xy2])

    new_data=new_data.sort_index()
    len(new_data)

    new_data=pd.concat([new_data,out_of_sample])


    len(new_data)

    data_X_y=new_data.copy()

    positive_per=data_X_y['label'][data_X_y['label']==1].sum()/len(data_X_y)
    positive_per

    pd.DataFrame([s.open,s.close]).T[-2:]

    

    classifiers = {
        'KN': KNeighborsClassifier(3),
        'SVC': SVC(kernel="linear", C=0.025),
        'SVC': SVC(gamma=2, C=1),
        'DT': DecisionTreeClassifier(max_depth=5),
        'RF': RandomForestClassifier(n_estimators=500),  # clf.feature_importances_
        'ET': ExtraTreesClassifier(n_estimators=500, max_depth=None),  # clf.feature_importances_
        'AB': AdaBoostClassifier(n_estimators=100),
        'GB': GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0), # clf.feature_importances_
        'GNB': GaussianNB(),
        'LD': LinearDiscriminantAnalysis(),
        'QD': QuadraticDiscriminantAnalysis()}

    data_X_y_copied=data_X_y.copy()

    len(data_X_y_copied)


    for name, clf in classifiers.items():

        print('\n\n=============  {}  =============='.format(name))
        time_mark=time.time()


        class_weight=1
        # rf_zl=RandomForestClassifier(n_estimators=500,class_weight={1:class_weight,-1:1},n_jobs=-1)
        # rf_zl=AdaBoostClassifier(LinearSVC(),n_estimators=500, learning_rate=1.0,algorithm='SAMME')
    #     rf_zl=GradientBoostingClassifier(n_estimators=600, learning_rate=0.7, max_depth=4, random_state=0)
        rf_zl=clf
        data_X_y = data_X_y_copied.copy()




        out_of_sample_test=1
        out_of_sample_test+=1
        n_splits=int(len(data_X_y)/out_of_sample_test)
        tscv = TimeSeriesSplit(n_splits=n_splits)


        # moving_test_times_of_future=22
        sum_right_count=0

        skip_previou_small_sample,count_recusive_times  =  0 ,0
        predict_up,predict_up_truely_up,actually_uprise=0,0,0
        predict_down,really_of_predict_down=0,0
        for train_i,test_j  in tscv.split(data_X_y):
            skip_previou_small_sample+=1
            if skip_previou_small_sample<=n_splits-moving_test_times_of_future:
                continue  # skip previou small sample

            count_recusive_times+=1   # count_recusive_times

        #     print('train={},test={}'.format(len(i),len(j)))
            train_X,test_X=data_X_y.iloc[train_i].iloc[:,:-2],data_X_y.iloc[test_j].iloc[:,:-2]
            train_y,test_y=data_X_y.iloc[train_i].iloc[:,-1],data_X_y.iloc[test_j].iloc[:,-1]
            rf_zl.fit(train_X,train_y)

            if int(rf_zl.predict(test_X))==1:
                predict_up+=1
                if int(test_y.values)==1:
                    predict_up_truely_up+=1
            elif int(rf_zl.predict(test_X))==-1:
                predict_down+=1
                if int(test_y.values)==-1:
                    really_of_predict_down+=1

            if int(test_y.values)==1:
                actually_uprise+=1

            sum_right_count += rf_zl.score(test_X,test_y)


        # dict_data_features={'features':train_X.columns,
        #            'feature_importace':rf_zl.feature_importances_}
        # df_feature_importace=pd.DataFrame(dict_data_features).set_index('features')
        # df_feature_importace=df_feature_importace.sort_values('feature_importace',ascending=False)



        print('accuracy  = {}'.format(sum_right_count/moving_test_times_of_future))

        if  predict_up!=0:
            print('predict_up_truely_up/predict_up   = {} ,predict_up= {} ,predict_up_truely_up = {}'.format(predict_up_truely_up/predict_up,predict_up,predict_up_truely_up))
    
        else:
            predict_up=np.nan
        
        print('predict none uprise of the stock/,actually uprise= {}'.format(actually_uprise))
        print('really_of_predict_down/predict_down = {}'.format(really_of_predict_down/predict_down))
        print('recusive time = {}'.format(count_recusive_times))

        print('slapse_time = {}'.format(round(time.time()-time_mark,5)))

        temp_temp=predict_up_truely_up/predict_up
        
#         if predict_up_truely_up not  in locals():
#             predict_up_truely_up=np.nan
#         if actually_uprise not  in locals():
#             actually_uprise=np.nan
#         if temp_temp not  in locals():
#             temp=np.nan
#         if predict_up not  in locals():
#             predict_up=np.nan
#         if (predict_up_truely_up/predict_up)==0
#              (predict_up_truely_up/predict_up)
        df_out.loc[len(df_out)]=[one_stock_name,name,temp_temp,predict_up,predict_up_truely_up,actually_uprise]
        print([one_stock_name,name,temp_temp,predict_up,predict_up_truely_up,actually_uprise])


    

# df_out=df_out.set_index('stock_name')

df_out



=============  RF  ==============
accuracy  = 0.6363636363636364
predict_up_truely_up/predict_up   = 0.14285714285714285 ,predict_up= 7 ,predict_up_truely_up = 1
predict none uprise of the stock/,actually uprise= 7
really_of_predict_down/predict_down = 0.7692307692307693
recusive time = 33
slapse_time = 35.17525
['002292.XSHE', 'RF', 0.14285714285714285, 7, 1, 7]


=============  KN  ==============
accuracy  = 0.42424242424242425
predict_up_truely_up/predict_up   = 0.2 ,predict_up= 20 ,predict_up_truely_up = 4
predict none uprise of the stock/,actually uprise= 7
really_of_predict_down/predict_down = 0.7692307692307693
recusive time = 33
slapse_time = 0.18481
['002292.XSHE', 'KN', 0.2, 20, 4, 7]


=============  SVC  ==============
accuracy  = 0.6666666666666666
predict_up_truely_up/predict_up   = 0.16666666666666666 ,predict_up= 6 ,predict_up_truely_up = 1
predict none uprise of the stock/,actually uprise= 7
really_of_predict_down/predict_down = 0.7777777777777778
recusive time = 33


/srv/env/lib64/python3.4/site-packages/pandas/core/algorithms.py:1225: RuntimeWarning:

invalid value encountered in subtract



accuracy  = 0.6060606060606061
predict_up_truely_up/predict_up   = 0.4 ,predict_up= 10 ,predict_up_truely_up = 4
predict none uprise of the stock/,actually uprise= 11
really_of_predict_down/predict_down = 0.6956521739130435
recusive time = 33
slapse_time = 35.03389
['300070.XSHE', 'RF', 0.4, 10, 4, 11]


=============  KN  ==============
accuracy  = 0.5151515151515151
predict_up_truely_up/predict_up   = 0.35294117647058826 ,predict_up= 17 ,predict_up_truely_up = 6
predict none uprise of the stock/,actually uprise= 11
really_of_predict_down/predict_down = 0.6875
recusive time = 33
slapse_time = 0.17841
['300070.XSHE', 'KN', 0.35294117647058826, 17, 6, 11]


=============  SVC  ==============
accuracy  = 0.7575757575757576
predict_up_truely_up/predict_up   = 1.0 ,predict_up= 3 ,predict_up_truely_up = 3
predict none uprise of the stock/,actually uprise= 11
really_of_predict_down/predict_down = 0.7333333333333333
recusive time = 33
slapse_time = 0.20992
['300070.XSHE', 'SVC', 1.0, 3, 3, 11

/srv/env/lib64/python3.4/site-packages/pandas/core/algorithms.py:1225: RuntimeWarning:

invalid value encountered in subtract



accuracy  = 0.6666666666666666
predict_up_truely_up/predict_up   = 0.42857142857142855 ,predict_up= 7 ,predict_up_truely_up = 3
predict none uprise of the stock/,actually uprise= 10
really_of_predict_down/predict_down = 0.7307692307692307
recusive time = 33
slapse_time = 35.74411
['002236.XSHE', 'RF', 0.42857142857142855, 7, 3, 10]


=============  KN  ==============
accuracy  = 0.3939393939393939
predict_up_truely_up/predict_up   = 0.2916666666666667 ,predict_up= 24 ,predict_up_truely_up = 7
predict none uprise of the stock/,actually uprise= 10
really_of_predict_down/predict_down = 0.6666666666666666
recusive time = 33
slapse_time = 0.17928
['002236.XSHE', 'KN', 0.2916666666666667, 24, 7, 10]


=============  SVC  ==============
accuracy  = 0.7272727272727273
predict_up_truely_up/predict_up   = 0.5714285714285714 ,predict_up= 7 ,predict_up_truely_up = 4
predict none uprise of the stock/,actually uprise= 10
really_of_predict_down/predict_down = 0.7692307692307693
recusive time = 33
sla

...

/srv/env/lib64/python3.4/site-packages/pandas/core/algorithms.py:1225: RuntimeWarning:

invalid value encountered in subtract



accuracy  = 0.6666666666666666
predict_up_truely_up/predict_up   = 0.2727272727272727 ,predict_up= 11 ,predict_up_truely_up = 3
predict none uprise of the stock/,actually uprise= 6
really_of_predict_down/predict_down = 0.8636363636363636
recusive time = 33
slapse_time = 35.84525
['002460.XSHE', 'RF', 0.2727272727272727, 11, 3, 6]


=============  KN  ==============
accuracy  = 0.5757575757575758
predict_up_truely_up/predict_up   = 0.2777777777777778 ,predict_up= 18 ,predict_up_truely_up = 5
predict none uprise of the stock/,actually uprise= 6
really_of_predict_down/predict_down = 0.9333333333333333
recusive time = 33
slapse_time = 0.18823
['002460.XSHE', 'KN', 0.2777777777777778, 18, 5, 6]


=============  SVC  ==============
accuracy  = 0.6666666666666666
predict_up_truely_up/predict_up   = 0.2727272727272727 ,predict_up= 11 ,predict_up_truely_up = 3
predict none uprise of the stock/,actually uprise= 6
really_of_predict_down/predict_down = 0.8636363636363636
recusive time = 33
slapse_

/srv/env/lib64/python3.4/site-packages/pandas/core/algorithms.py:1225: RuntimeWarning:

invalid value encountered in subtract



accuracy  = 0.6363636363636364
predict_up_truely_up/predict_up   = 0.45454545454545453 ,predict_up= 11 ,predict_up_truely_up = 5
predict none uprise of the stock/,actually uprise= 11
really_of_predict_down/predict_down = 0.7272727272727273
recusive time = 33
slapse_time = 35.41006
['000725.XSHE', 'RF', 0.45454545454545453, 11, 5, 11]


=============  KN  ==============
accuracy  = 0.6363636363636364
predict_up_truely_up/predict_up   = 0.45454545454545453 ,predict_up= 11 ,predict_up_truely_up = 5
predict none uprise of the stock/,actually uprise= 11
really_of_predict_down/predict_down = 0.7272727272727273
recusive time = 33
slapse_time = 0.18698
['000725.XSHE', 'KN', 0.45454545454545453, 11, 5, 11]


=============  SVC  ==============
accuracy  = 0.6363636363636364
predict_up_truely_up/predict_up   = 0.42857142857142855 ,predict_up= 7 ,predict_up_truely_up = 3
predict none uprise of the stock/,actually uprise= 11
really_of_predict_down/predict_down = 0.6923076923076923
recusive time = 3

,stock_name,clf_name,predict_up_truely_up/predict_up,predict_up,predict_up_truely_up,actually_uprise
0,002292.XSHE,RF,0.142857,7.0,1.0,7.0
1,002292.XSHE,KN,0.200000,20.0,4.0,7.0
2,002292.XSHE,SVC,0.166667,6.0,1.0,7.0
3,002292.XSHE,DT,0.166667,6.0,1.0,7.0
4,002292.XSHE,LD,0.000000,3.0,0.0,7.0
5,002292.XSHE,ET,0.166667,6.0,1.0,7.0
6,002292.XSHE,GNB,0.333333,3.0,1.0,7.0
7,002292.XSHE,GB,0.000000,3.0,0.0,7.0
8,002292.XSHE,QD,0.000000,1.0,0.0,7.0
9,002292.XSHE,AB,0.000000,3.0,0.0,7.0


In [6]:
df_out

,stock_name,clf_name,predict_up_truely_up/predict_up,predict_up,predict_up_truely_up,actually_uprise
0,002292.XSHE,RF,0.142857,7.0,1.0,7.0
1,002292.XSHE,KN,0.200000,20.0,4.0,7.0
2,002292.XSHE,SVC,0.166667,6.0,1.0,7.0
3,002292.XSHE,DT,0.166667,6.0,1.0,7.0
4,002292.XSHE,LD,0.000000,3.0,0.0,7.0
5,002292.XSHE,ET,0.166667,6.0,1.0,7.0
6,002292.XSHE,GNB,0.333333,3.0,1.0,7.0
7,002292.XSHE,GB,0.000000,3.0,0.0,7.0
8,002292.XSHE,QD,0.000000,1.0,0.0,7.0
9,002292.XSHE,AB,0.000000,3.0,0.0,7.0


In [22]:
df_out.to_csv('last10.csv')